In [1]:
from herpetologist import check_type
from typeguard import typechecked

In [2]:
class A:
    def __init__(self):
        pass
    
    @check_type
    def pur(self, a: str):
        print(a)

In [3]:
a = A()

In [7]:
a.pur('a')

a


In [9]:
def b(c):
    a.pur(c)
    
b(1)

TypeError: type of argument "a" must be 'str'

In [5]:
@check_type
def greeting(name: str):
    print(name)
    
greeting('foo') # no error
greeting(1) # error

foo


TypeError: type of argument "name" must be 'str'

In [6]:
from typing import List, Dict, Tuple

In [7]:
@check_type
def greeting(name: List[Dict[str, Dict[str, List[str]]]]):
    print(name)
    
greeting([{'a': {'b': ['a']}}])

[{'a': {'b': ['a']}}]


In [8]:
@typechecked
def greeting(name: str):
    pass

greeting('husein') # no error
greeting(1) # error

TypeError: type of argument "name" must be str; got int instead

In [9]:
from typing import List, Tuple, Dict

In [10]:
@check_type
def greeting(name: List[str]):
    pass

greeting(['h']) # no error
greeting([1]) # error

TypeError: type of argument "name" must be List[str]

In [11]:
@typechecked
def greeting(name: List[str]):
    pass

greeting(['h']) # no error
greeting([1]) # error

TypeError: type of argument "name"[0] must be str; got int instead

In [12]:
@check_type
def greeting(name: List[Tuple[str, int, float]]):
    pass

greeting([('h', 1, 1.1)]) # no error
greeting([('h', 1, 1)]) # error

TypeError: type of argument "name" must be List[Tuple[str, int, float]]

In [13]:
@typechecked
def greeting(name: List[Tuple[str, int, float]]):
    pass

greeting([('h', 1, 1.1)]) # no error
greeting([('h', 1, 1)]) # no error, supposedly error

In [14]:
@check_type
def greeting(name: List[Tuple[str, int, Dict[int, int]]]):
    pass

greeting([('h', 1, {1:2})]) # no error
greeting([('h', 1, {1:2.2})]) # error

TypeError: type of argument "name" must be List[Tuple[str, int, Dict[int, int]]]

In [15]:
@check_type
def greeting(name: Tuple[Tuple[List[Dict[int, str]]], Tuple[List[int]]]):
    pass

greeting((([{1: 'str'}],), ([1], [1]))) # no error
greeting((([{1: 'str'}],), ([1], [1.1]))) # error

TypeError: type of argument "name" must be Tuple[Tuple[List[Dict[int, str]]], Tuple[List[int]]]

In [16]:
@check_type
def greeting(name: List[Dict[str, Dict[str, List[str]]]]):
    pass
    
greeting([{'a': {'b': ['a', 'a']}}]) # no error
greeting([{'a': {'b': ['a', 1]}}]) # error

TypeError: type of argument "name" must be List[Dict[str, Dict[str, List[str]]]]

In [17]:
from test_class import A

In [18]:
a = A()

In [19]:
@check_type
def greeting(name: Tuple[Tuple[List[Dict[int, A]]], Tuple[List[int], float, Tuple[Dict[str, str]]]]):
    pass

greeting((([{1: a}],), ([1], 1.1, ({'a': 'a'},)))) # no error
greeting((([{1: 'str'}],), ([1], 1.1, ({'a': 'a'}, {'a': 1})))) # error

TypeError: type of argument "name" must be Tuple[Tuple[List[Dict[int, test_class.A]]], Tuple[List[int], float, Tuple[Dict[str, str]]]]

## default value

herpetologist will never checked default value, snake is a snake (concept from duck typing)!

In [20]:
@check_type
def greeting(name: List[Dict[str, Dict[str, List[str]]]] = None):
    if name is None:
        print('name is None!')
    
greeting([{'a': {'b': ['a', 'a']}}]) # no error
greeting() # no error

name is None!
